In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import backtrader as bt
import yfinance as yf
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Download historical data
start_date = "2023-01-01"
end_date =  "2023-10-31"

cerebro = bt.Cerebro()
for symbol in ['QQQ', 'SPY', 'GLD']:
    data = bt.feeds.PandasData(dataname=yf.download(symbol, start=start_date, end=end_date))
    cerebro.adddata(data, name=symbol)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [31]:
df=yf.download('QQQ', start=start_date, end=end_date)
df.isnull().sum()


[*********************100%***********************]  1 of 1 completed


Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [4]:
from collections import defaultdict
class IndicatorObserver(bt.observer.Observer):
    lines = ('qqq_ma', 'spy_fast_ma', 'spy_slow_ma')
    plotlines = dict(
        qqq_ma=dict(color='blue'),
        spy_fast_ma=dict(color='green'),
        spy_slow_ma=dict(color='red')
    )

    def next(self):
        self.lines.qqq_ma[0] = self.strategy.qqq_ma[0]
        self.lines.spy_fast_ma[0] = self.strategy.spy_fast_ma[0]
        self.lines.spy_slow_ma[0] = self.strategy.spy_slow_ma[0]


In [46]:
class PrintClose(bt.Strategy):

    def __init__(self):
        #Keep a reference to the "close" line in the data[0] dataseries
        self.qqq = self.datas[0]
        self.spy = self.datas[1]
        self.gld = self.datas[2]

        self.qqq_ma = bt.indicators.SimpleMovingAverage(self.qqq, period=30)
        self.spy_fast_ma = bt.indicators.SimpleMovingAverage(self.spy, period=50)
        self.spy_slow_ma = bt.indicators.SimpleMovingAverage(self.spy, period=200)

        self.qqq_cross = bt.indicators.CrossOver(self.qqq, self.qqq_ma)
        self.spy_cross = bt.indicators.CrossOver(self.spy_fast_ma, self.spy_slow_ma)

    def log(self, txt, value1=None, value2=None, value3=None):
        dt = self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt} {value1} {value2} {value3}') 

    def next(self):
        self.log('spy_cross: ', self.spy_cross[0], self.spy_fast_ma[0], self.spy_slow_ma[0]) 
        self.log('qqq_cross: ', self.qqq_cross[0], self.qqq_ma[0], self.qqq[0]) 

cerebro = bt.Cerebro()

start_date = "2022-01-01"
end_date =  "2023-10-31"
for symbol in ['QQQ', 'SPY', 'GLD']:
    data = bt.feeds.PandasData(dataname=yf.download(symbol, start=start_date, end=end_date))
    cerebro.adddata(data, name=symbol)

#Add strategy to Cerebro
cerebro.addstrategy(PrintClose)

#Run Cerebro Engine
cerebro.run()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
2022-10-19 spy_cross:  0.0 389.8696014404297 413.51225128173826
2022-10-19 qqq_cross:  0.0 279.81800231933596 270.489990234375
2022-10-20 spy_cross:  0.0 388.7780017089844 412.951551361084
2022-10-20 qqq_cross:  0.0 278.7710021972656 269.1099853515625
2022-10-21 spy_cross:  0.0 387.8640020751953 412.48110137939454
2022-10-21 qqq_cross:  0.0 277.7153361002604 275.4200134277344
2022-10-24 spy_cross:  0.0 386.89940185546874 412.03575134277344
2022-10-24 qqq_cross:  1.0 276.639003499349 278.45001220703125
2022-10-25 spy_cross:  0.0 386.0206024169922 411.62990142822264
2022-10-25 qqq_cross:  0.0 276.3226694742838 284.2099914550781
2022-10-26 spy_cross:  0.0 385.067001953125 411.2124513244629
2022-10-26 qqq_cross:  0.0 275.7193359375 277.92999267578125
2022-10-27 spy_cross:  0.0 384.13360

In [62]:
class QuantStrategy(bt.Strategy):
    params = (
        ('qqq_ma_period', 30),
        ('spy_fast_ma', 50),
        ('spy_slow_ma', 200),
    )

    def __init__(self):
        self.qqq = self.datas[0]
        self.spy = self.datas[1]
        self.gld = self.datas[2]

        self.qqq_ma = bt.indicators.SimpleMovingAverage(self.qqq, period=self.p.qqq_ma_period)
        self.spy_fast_ma = bt.indicators.SimpleMovingAverage(self.spy, period=self.p.spy_fast_ma)
        self.spy_slow_ma = bt.indicators.SimpleMovingAverage(self.spy, period=self.p.spy_slow_ma)

        self.qqq_cross = bt.indicators.CrossOver(self.qqq, self.qqq_ma)
        self.spy_cross = bt.indicators.CrossOver(self.spy_fast_ma, self.spy_slow_ma)



    def next(self):
        if not self.position:
            if self.qqq_cross > 0:  # Nasdaq 100 crosses above 30-day MA
                    
                # Calculate position size based on portfolio value
                portfolio_value = self.broker.getvalue()
                price = self.qqq.close[0]
                shares = (portfolio_value) // price 

                self.buy(data=self.qqq, size=shares)
                #self.buy(data=self.qqq)
                print(str(self.datas[0].datetime.date(0))+" bought QQQ")
                
            elif self.spy_cross > 0:  # S&P 500 Golden Cross
                portfolio_value = self.broker.getvalue()
                price = self.spy.close[0]
                shares = (portfolio_value) // price 

                self.buy(data=self.spy, size=shares)
                #self.buy(data=self.spy)
                print(str(self.datas[0].datetime.date(0))+" bought SPY")
        else:
            if self.getposition(self.qqq).size > 0:
                if self.qqq_cross < 0:  # Nasdaq 100 crosses below 30-day MA
                    self.close(data=self.qqq)

                    portfolio_value = self.broker.getvalue()
                    price = self.gld.close[0]
                    shares = (portfolio_value) // price 

                    self.buy(data=self.gld, size=shares)
                    #self.buy(data=self.gld)

                    print(str(self.datas[0].datetime.date(0))+" sold QQQ in exchange for GLD")
                    #print("Sold QQQ position: "+ str(self.get_fundshares(self.qqq)))
            elif self.getposition(self.spy).size > 0:
                if self.spy_cross < 0:  # S&P 500 Death Cross
                    self.close(data=self.spy)

                    portfolio_value = self.broker.getvalue()
                    price = self.gld.close[0]
                    shares = (portfolio_value) // price 

                    self.buy(data=self.gld, size=shares)(data=self.gld)
                    print(str(self.datas[0].datetime.date(0))+" sold SPY in exchange for GLD")
            elif self.getposition(self.gld).size > 0:
                if self.qqq_cross > 0:
                    self.close(data=self.gld)
                    portfolio_value = self.broker.getvalue()
                    price = self.qqq.close[0]
                    shares = (portfolio_value) // price 

                    self.buy(data=self.qqq, size=shares)
                    print(str(self.datas[0].datetime.date(0))+" sold GLD in exchange for QQQ")
                elif self.spy_cross > 0:
                    self.close(data=self.gld)
                    portfolio_value = self.broker.getvalue()
                    price = self.spy.close[0]
                    shares = (portfolio_value) // price 

                    self.buy(data=self.spy, size=shares)
                    print(str(self.datas[0].datetime.date(0))+" sold GLD in exchange for SPY")



In [63]:
cerebro = bt.Cerebro()

start_date = "2022-01-01"
end_date =  "2023-10-31"
for symbol in ['QQQ', 'SPY', 'GLD']:
    data = bt.feeds.PandasData(dataname=yf.download(symbol, start=start_date, end=end_date))
    cerebro.adddata(data, name=symbol)



cerebro.addstrategy(QuantStrategy)

# cerebro.addobserver(IndicatorObserver)

# Set initial capital
cerebro.broker.setcash(100000)
cerebro.broker.set_fundmode(True)
# Add analyzers
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.TimeReturn, fund=True)  # track fund value

# Run the backtest
results = cerebro.run()

# Print results
print(f"Final Portfolio Value: ${cerebro.broker.getvalue():.2f}")
#print(f"Sharpe Ratio: {results[0].analyzers.sharpe_ratio.get_analysis()['sharperatio']:.2f}")
print(f"Max Drawdown: {results[0].analyzers.drawdown.get_analysis()['max']['drawdown']:.2f}%")
print(f"Total Return: {results[0].analyzers.returns.get_analysis()['rtot']:.2f}%")

# Plot the results
# Then plot with explicit show
cerebro.plot()



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
2022-10-24 bought QQQ
2022-10-28 bought QQQ
2022-11-02 sold QQQ in exchange for GLD
2022-11-10 bought QQQ
2023-01-11 bought QQQ
2023-02-02 bought SPY
2023-03-03 bought QQQ
2023-03-16 bought QQQ
2023-04-27 bought QQQ
2023-05-05 bought QQQ
2023-08-07 bought QQQ
2023-08-29 bought QQQ
2023-10-10 bought QQQ
2023-10-16 bought QQQ
Final Portfolio Value: $115851.95
Max Drawdown: 11.33%
Total Return: 0.15%


<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 10 Axes>]]

In [26]:
results[0]